## Encoding of sd-shell $H_Q$ for quantum annealing

## Encoding of sd shell $H$ for quantum annealing

Imports

In [2]:
from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np

from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverterOnlynnpp,HardcoreBosonsBasis
from scipy.sparse import lil_matrix


file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)



nparts=[(2,0)]
titles=[r'$^{18}$O']



size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{18}$O'

Two-body matrix

In [3]:
twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

Computing the matrix, pls wait... (u_u) 



100%|██████████| 24/24 [00:50<00:00,  2.09s/it]


Nuclear Shell Model Hamiltonian

In [4]:
from scipy.sparse import identity
#Quadrupole Operator
energy_errors=[]
fidelities=[]
for idx,npart in enumerate(nparts):
    nparticles_a,nparticles_b=npart
    title=titles[idx]
    
    # compute the NSM Hamiltonian
    NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
    print('size=',size_a+size_b,size_b)
    NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
    if file_name=='data/cki':
        twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

        NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
    else:
        NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
    NSMHamiltonian.get_hamiltonian()

    
    shifted_hamiltonian=NSMHamiltonian.hamiltonian


    egs,psi0=eigsh(shifted_hamiltonian,k=1,which='SA')

    print('ground state energy=',egs[0])

size= 24 12
ground state energy= -11.931788251561525


In [5]:
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper,BravyiKitaevMapper

# Start building terms
hamiltonian_terms = {}
n_spin_orbitals =  size_a  # Assuming spin-orbitals are double the single-particle states
# One-body terms: e_a * c†_a c_a
for a, e_a in enumerate(SPS.energies[:size_a]):
    label = f"+_{a} -_{a}"
    hamiltonian_terms[label] = hamiltonian_terms.get(label, 0) + e_a

# Two-body terms: (1/4) * v_abcd * c†_a c†_b c_d c_c
for (a, b, c, d), v_abcd in twobody_matrix.items():
    if a<size_a and b<size_a and c<size_a and d<size_a:
        label = f"+_{a} +_{b} -_{d} -_{c}"
        coef = 0.25 * v_abcd
        hamiltonian_terms[label] = hamiltonian_terms.get(label, 0) + coef

# Create FermionicOp
nsm_hamiltoninan_fermions = FermionicOp(hamiltonian_terms, num_spin_orbitals=n_spin_orbitals)

# (Optional) Print to inspect
print(nsm_hamiltoninan_fermions)

Fermionic Operator
number spin orbitals=12, number terms=2492
  -3.9257 * ( +_0 -_0 )
+ -3.9257 * ( +_1 -_1 )
+ -3.9257 * ( +_2 -_2 )
+ -3.9257 * ( +_3 -_3 )
+ -3.9257 * ( +_4 -_4 )
+ -3.9257 * ( +_5 -_5 )
+ -3.2079 * ( +_6 -_6 )
+ -3.2079 * ( +_7 -_7 )
+ 2.1117 * ( +_8 -_8 )
+ 2.1117 * ( +_9 -_9 )
+ 2.1117 * ( +_10 -_10 )
+ 2.1117 * ( +_11 -_11 )
+ -0.05172500000000001 * ( +_0 +_1 -_1 -_0 )
+ 0.05172500000000001 * ( +_0 +_1 -_0 -_1 )
+ 0.05172500000000001 * ( +_1 +_0 -_1 -_0 )
+ -0.05172500000000001 * ( +_1 +_0 -_0 -_1 )
+ 0.3337250000000001 * ( +_0 +_1 -_8 -_0 )
+ -0.3337250000000001 * ( +_0 +_1 -_0 -_8 )
+ -0.3337250000000001 * ( +_1 +_0 -_8 -_0 )
+ 0.3337250000000001 * ( +_1 +_0 -_0 -_8 )
+ -0.05172500000000001 * ( +_0 +_2 -_2 -_0 )
+ 0.05172500000000001 * ( +_0 +_2 -_0 -_2 )
+ 0.05172500000000001 * ( +_2 +_0 -_2 -_0 )
+ -0.05172500000000001 * ( +_2 +_0 -_0 -_2 )
+ 0.20436399110257913 * ( +_0 +_2 -_9 -_0 )
+ -0.20436399110257913 * ( +_0 +_2 -_0 -_9 )
+ -0.20436399110257913 * ( +_2 

In [6]:
mapper = JordanWignerMapper()
nsm_hamiltonian_qubit = mapper.map(nsm_hamiltoninan_fermions)

In [7]:
print(NSMHamiltonian.hamiltonian)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 180 stored elements and shape (14, 14)>
  Coords	Values
  (0, 0)	-9.3151
  (0, 1)	0.6897999999999996
  (0, 2)	0.20328479938540372
  (0, 3)	-0.4062999999999996
  (0, 4)	-0.5082840208321855
  (0, 5)	0.18949464964372487
  (0, 6)	0.5082840208321854
  (0, 7)	0.18949464964372495
  (0, 8)	0.20328479938540372
  (0, 9)	-0.900781889989654
  (0, 10)	0.48367995370807726
  (0, 11)	0.48367995370807726
  (0, 12)	-1.9314456782406446
  (0, 13)	0.6017349640876247
  (1, 0)	0.6897999999999996
  (1, 1)	-8.8615
  (1, 2)	0.30763695987708084
  (1, 3)	0.8598999999999998
  (1, 4)	-0.10165680416643715
  (1, 5)	0.6918637014569979
  (1, 6)	0.10165680416643713
  (1, 7)	0.691863701456998
  (1, 8)	0.30763695987708073
  (1, 9)	0.900781889989654
  (1, 10)	0.09673599074161551
  :	:
  (12, 3)	-0.7347060355029268
  (12, 4)	-0.15865000000000007
  (12, 5)	-0.09509586140912163
  (12, 6)	0.15865000000000004
  (12, 7)	-0.09509586140912159
  (12, 8)	0.232936337102774

In [8]:

from scipy.sparse import csr_matrix
n_qubits = nsm_hamiltonian_qubit.num_qubits

for idx_b,b in enumerate(NSMHamiltonian.basis):
    for idx_d,d in enumerate(NSMHamiltonian.basis):
        b_1,b_2=np.nonzero(b)[0]
        d_1,d_2=np.nonzero(d)[0]

        print(f'i={(idx_b)},j={(idx_d)} ->{csr_matrix(nsm_hamiltonian_qubit.to_matrix())[2**(b_1)+2**(b_2),2**(d_1)+2**(d_2)]-NSMHamiltonian.hamiltonian[idx_b,idx_d]} \n')


i=0,j=0 ->(8.881784197001252e-15+0j) 

i=0,j=1 ->(2.220446049250313e-16+0j) 

i=0,j=2 ->(-2.7755575615628914e-17+0j) 

i=0,j=3 ->(1.1102230246251565e-16+0j) 

i=0,j=4 ->0j 

i=0,j=5 ->0j 

i=0,j=6 ->0j 

i=0,j=7 ->(-5.551115123125783e-17+0j) 

i=0,j=8 ->(-2.7755575615628914e-17+0j) 

i=0,j=9 ->(1.1102230246251565e-16+0j) 

i=0,j=10 ->(5.551115123125783e-17+0j) 

i=0,j=11 ->(5.551115123125783e-17+0j) 

i=0,j=12 ->(2.220446049250313e-16+0j) 

i=0,j=13 ->0j 

i=1,j=0 ->(2.220446049250313e-16+0j) 

i=1,j=1 ->(7.105427357601002e-15+0j) 

i=1,j=2 ->(-1.6653345369377348e-16+0j) 

i=1,j=3 ->0j 

i=1,j=4 ->(-2.7755575615628914e-17+0j) 

i=1,j=5 ->0j 

i=1,j=6 ->(4.163336342344337e-17+0j) 

i=1,j=7 ->0j 

i=1,j=8 ->0j 

i=1,j=9 ->(-1.1102230246251565e-16+0j) 

i=1,j=10 ->(1.3877787807814457e-17+0j) 

i=1,j=11 ->(1.3877787807814457e-17+0j) 

i=1,j=12 ->0j 

i=1,j=13 ->0j 

i=2,j=0 ->(-2.7755575615628914e-17+0j) 

i=2,j=1 ->(-1.6653345369377348e-16+0j) 

i=2,j=2 ->(2.6645352591003757e-15+0j) 

i=2

Driver Hamiltonian

In [17]:
#### Initial Hamiltonian
min_b=np.zeros(size_a+size_b)

if file_name=='data/cki':
    order_of_filling=np.asarray([0,3,1,2,4,5])
    order_of_filling_protons=order_of_filling+size_a
else:
    order_of_filling=np.asarray([0,5,1,4,2,3,6,7,8,11,9,10])
    order_of_filling_protons=order_of_filling+size_a


min_b[order_of_filling[:nparticles_a]]=1
min_b[order_of_filling_protons[:nparticles_b]]=1

print(min_b)

print('initial state=',min_b)
InitialHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
kinetic_term:Dict={}
adj_matrix=np.zeros((size_a+size_b,size_a+size_b))
idx=InitialHamiltonian._get_index(element=min_b)
print('idx=',idx)
psi_configuration=np.zeros(NSMHamiltonian.hamiltonian.shape[0])
psi_configuration[idx]=1
min=psi_configuration.transpose().dot(NSMHamiltonian.hamiltonian.dot(psi_configuration))      
external_field=np.zeros(size_a+size_b)
external_field=-1*(np.abs(min)/(nparticles_a+nparticles_b))*min_b
print('min energy=',min)
InitialHamiltonian.get_external_potential(external_field)
InitialHamiltonian.get_hamiltonian()

print('external field=',external_field)

e0,psis=InitialHamiltonian.get_spectrum(n_states=1)
psi_configuration=psis[:,0]



[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
initial state= [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
idx= 0
min energy= -9.3151
external field= [-4.65755 -0.      -0.      -0.      -0.      -4.65755 -0.      -0.
 -0.      -0.      -0.      -0.      -0.      -0.      -0.      -0.
 -0.      -0.      -0.      -0.      -0.      -0.      -0.      -0.     ]


In [23]:
print(InitialHamiltonian.hamiltonian)

  (0, 0)	-9.3151


In [18]:
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper,BravyiKitaevMapper

# Start building terms
hamiltonian_terms = {}
n_spin_orbitals =  size_a  # Assuming spin-orbitals are double the single-particle states
# One-body terms: e_a * c†_a c_a
for a, e_a in enumerate(external_field[:size_a]):
    label = f"+_{a} -_{a}"
    hamiltonian_terms[label] = hamiltonian_terms.get(label, 0) + e_a


# Create FermionicOp
driver_hamiltoninan_fermions = FermionicOp(hamiltonian_terms, num_spin_orbitals=n_spin_orbitals)

# (Optional) Print to inspect
print(driver_hamiltoninan_fermions)

Fermionic Operator
number spin orbitals=12, number terms=12
  -4.65755 * ( +_0 -_0 )
+ 0.0 * ( +_1 -_1 )
+ 0.0 * ( +_2 -_2 )
+ 0.0 * ( +_3 -_3 )
+ 0.0 * ( +_4 -_4 )
+ -4.65755 * ( +_5 -_5 )
+ 0.0 * ( +_6 -_6 )
+ 0.0 * ( +_7 -_7 )
+ 0.0 * ( +_8 -_8 )
+ 0.0 * ( +_9 -_9 )
+ 0.0 * ( +_10 -_10 )
+ 0.0 * ( +_11 -_11 )


In [19]:
mapper = JordanWignerMapper()
driver_hamiltoninan_qubit = mapper.map(driver_hamiltoninan_fermions)

In [20]:
from scipy.sparse import csr_matrix
n_qubits = driver_hamiltoninan_qubit.num_qubits

for idx_b,b in enumerate(InitialHamiltonian.basis):

    b_1,b_2=np.nonzero(b)[0]

    print(f'i={(idx_b)},j={(idx_b)} ->{csr_matrix(driver_hamiltoninan_qubit.to_matrix())[2**(b_1)+2**(b_2),2**(b_1)+2**(b_2)]-InitialHamiltonian.hamiltonian[idx_b,idx_b]} \n')

i=0,j=0 ->0j 

i=1,j=1 ->0j 

i=2,j=2 ->0j 

i=3,j=3 ->0j 

i=4,j=4 ->0j 

i=5,j=5 ->0j 

i=6,j=6 ->0j 

i=7,j=7 ->0j 

i=8,j=8 ->0j 

i=9,j=9 ->0j 

i=10,j=10 ->0j 

i=11,j=11 ->0j 

i=12,j=12 ->0j 

i=13,j=13 ->0j 



Time evolution

In [21]:
from qiskit.quantum_info import SparsePauliOp
from qiskit.synthesis import SuzukiTrotter,QDrift
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import transpile
from qiskit.quantum_info import Statevector

import qiskit
import numpy as np
from qiskit.quantum_info import SparsePauliOp

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from typing import Dict
from tqdm import tqdm


##### Counting for one time step

In [22]:
#final_state=Statevector(circuit_opt)
time_steps=1
tf=0.2
time=np.linspace(0.1,tf,time_steps)
dt=tf/time_steps
print(dt)
#b=0.8
#h=1-(1+b)*(time/tf)+b*(time/tf)**2
driver=1-time/tf
circuit_time_evolution=QuantumCircuit(nsm_hamiltonian_qubit.num_qubits)
circuit_time_evolution.x([0]) # initial state

energies_qiskit=np.zeros((time_steps))
tbar=tqdm(enumerate(time))

for n,t in tbar:
    hamiltonian_t=(driver[n])*nsm_hamiltonian_qubit+(1-driver[n])*driver_hamiltoninan_qubit
    exp_H_t=PauliEvolutionGate(hamiltonian_t,time=dt,synthesis=SuzukiTrotter(order=2))
    #exp_H_t=PauliEvolutionGate(hamiltonian_t,time=dt,synthesis=QDrift(reps=5))
    circuit_time_evolution.append(exp_H_t,range(nsm_hamiltonian_qubit.num_qubits))
    single_particle_vector=np.zeros(2**n_qubits,dtype=np.complex128)
    # psi_for_fidelity=np.zeros(n_qubits,dtype=np.complex128)
    # for a in range(n_qubits):
    #     a_mb=2**(a)
    #     final_state=Statevector(circuit_time_evolution).data
    #     single_particle_vector[a_mb]=final_state[a_mb]
    #     psi_for_fidelity[a]=final_state[a_mb]
    # energies_qiskit[n]=Statevector(single_particle_vector).expectation_value(hamiltonian_t).real
    tbar.refresh()
final_state=Statevector(circuit_time_evolution)


0.2


1it [00:00, 92.73it/s]


In [23]:
transpiled_circuit_time_evolution=transpile(circuit_time_evolution.decompose(), optimization_level=3,basis_gates=['cx','s','h','rz','x'])


In [24]:

print(
    f"""
Trotter step with Suzuki Trotter (1nd order)
--------------------------------------------

                  Depth: {transpiled_circuit_time_evolution.depth()}
             Gate count: {len(transpiled_circuit_time_evolution)}
    Nonlocal gate count: {transpiled_circuit_time_evolution.num_nonlocal_gates()}
         Gate breakdown: {", ".join([f"{k.upper()}: {v}" for k, v in transpiled_circuit_time_evolution.count_ops().items()])}

"""
)



# And finall
%matplotlib inline
#initial_state.draw("text")


Trotter step with Suzuki Trotter (1nd order)
--------------------------------------------

                  Depth: 79981
             Gate count: 133060
    Nonlocal gate count: 34390
         Gate breakdown: RZ: 68931, CX: 34390, H: 27386, S: 2353




In [54]:
#final_state=Statevector(circuit_opt)
time_steps=100
tf=10.
time=np.linspace(0.1,tf,time_steps)
dt=tf/time_steps
print(dt)
#b=0.8
#h=1-(1+b)*(time/tf)+b*(time/tf)**2
driver=1-time/tf
circuit_time_evolution=QuantumCircuit(nsm_hamiltonian_qubit.num_qubits)
circuit_time_evolution.x([0]) # initial state
circuit_time_evolution.x([5])

energies_qiskit=np.zeros((time_steps))
tbar=tqdm(enumerate(time))

for n,t in tbar:
    hamiltonian_t=(1-driver[n])*nsm_hamiltonian_qubit+(driver[n])*driver_hamiltoninan_qubit
    exp_H_t=PauliEvolutionGate(hamiltonian_t,time=dt,synthesis=SuzukiTrotter(order=2))
    #exp_H_t=PauliEvolutionGate(hamiltonian_t,time=dt,synthesis=QDrift(reps=5))
    circuit_time_evolution.append(exp_H_t,range(nsm_hamiltonian_qubit.num_qubits))
    #single_particle_vector=np.zeros(2**n_qubits,dtype=np.complex128)
    # psi_for_fidelity=np.zeros(n_qubits,dtype=np.complex128)
    # for a in range(n_qubits):
    #     a_mb=2**(a)
    #     final_state=Statevector(circuit_time_evolution).data
    #     single_particle_vector[a_mb]=final_state[a_mb]
    #     psi_for_fidelity[a]=final_state[a_mb]
    # energies_qiskit[n]=Statevector(single_particle_vector).expectation_value(hamiltonian_t).real
    tbar.refresh()
final_state=Statevector(circuit_time_evolution)

0.1


100it [00:01, 73.83it/s]


Compute the energy

In [55]:
from qiskit.quantum_info import state_fidelity,Statevector

print(final_state.expectation_value(nsm_hamiltonian_qubit).real)


-11.88876870992596


#### Counting CNOT

In [ ]:
n_cnot_nsm=34390
#### We focus on the fidelity
titles=[r'$^{18}$O',r'$^{20}$O',r'$^{22}$O',r'$^{20}$Ne', r'$^{22}$Ne',r'$^{24}$Ne',r'$^{26}$Ne',r'$^{24}$Mg',r'$^{26}$Mg',r'$^{28}$Mg',r'$^{26}$Si',]
values=[n_cnot_nsm*100,n_cnot_nsm*30,4*n_cnot_nsm,3*n_cnot_nsm,15*n_cnot_nsm, 13*n_cnot_nsm,15*n_cnot_nsm,5*n_cnot_nsm,10*n_cnot_nsm,n_cnot_nsm,]